**Mounting Drives**


In [2]:
from google.colab import files, drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Ridge Regression Analysis**


In [80]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import math

url = '/content/drive/My Drive/input/wx-frontier-agg.csv'
dataframe = pd.read_csv(url, index_col=0)
data = dataframe.values

## I think this is where my errors are coming from; what is train_X and train_Y
X, y = data[:, : -1], data[:, -1]

# Define Ridge Model
ridge_Model = Ridge(alpha=1.0)

# Five-fold cross Validation
cvScoreRidge = cross_val_score(ridge_Model, X, y, scoring="neg_mean_squared_error", cv=5)
cvScoreRidge = np.absolute(cvScoreRidge)
cvScoreRidge = math.sqrt(cvScoreRidge.mean())

## RMSE Scores
print("Ridge Regression RMSE Score {}".format(round(cvScoreRidge, 3)))

Ridge Regression RMSE Score 1.209


**Random Forest**


In [81]:
from sklearn import tree
from sklearn import ensemble

# Define Random Forest Model
RF_model = ensemble.RandomForestClassifier()

# Five Fold Cross Validation
cvScoreRF = cross_val_score(RF_model, X, y, scoring="neg_mean_squared_error", cv=5)
cvScoreRF = np.absolute(cvScoreRF)
cvScoreRF = math.sqrt(cvScoreRF.mean())

## RMSE Scores
print("Random Forest RMSE Score {}".format(round(cvScoreRF, 3)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Random Forest RMSE Score 1.799
